# Thinking in Coroutines

![image](https://raw.githubusercontent.com/baojiweicn/Thinking-in-Coroutines/master/imgs/coroutines_example.png)

![image](https://raw.githubusercontent.com/baojiweicn/Thinking-in-Coroutines/master/imgs/thread_example.png)

In [ ]:
import asyncio

if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.runforever()

# More Example

In [ ]:
import asyncio
import datetime


def anything(i):
    print(i, datetime.datetime.now())


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.call_later(2, loop.stop)
    for i in range(1, 4):
        loop.call_soon(anything, i)
    try:
        loop.run_forever()
    finally:
        loop.close()

![image](https://raw.githubusercontent.com/baojiweicn/Thinking-in-Coroutines/master/imgs/async_callbacks.png)

In [ ]:
import datetime
import asyncio
import time


def anything(i):
    print(i, datetime.datetime.now())
    time.sleep(i)


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.call_later(2, loop.stop)
    for i in range(1, 4):
        loop.call_soon(anything, i)
    try:
        loop.run_forever()
    finally:
        loop.close()
        print('close', datetime.datetime.now())

In [ ]:
# -*- coding: utf-8 -*-

import asyncio
import datetime


async def anything(i):
    print(i, datetime.datetime.now())
    await asyncio.sleep(i)


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.call_later(2, loop.stop)
    for i in range(1, 4):
        loop.create_task(anything(i))
    try:
        loop.run_forever()
    finally:
        loop.close()
        print('close', datetime.datetime.now())

![image](https://github.com/baojiweicn/Thinking-in-Coroutines/blob/master/imgs/event_loop.png?raw=true)

In [ ]:
from asyncio import futures


class Task(futures.Future):
    def __init__(self, loop=None):
        super().__init__(loop=loop)
        self.count = 0
        self._loop.call_soon(self._step)

    def _step(self):
        try:
            result = next(self._coro)
        except StopIteration as exc:
            self.set_result(exc.value)
        except BaseException as exc:
            self.set_exception(exc)
        else:
            self.count += 1
            self._loop.call_soon(self._step)

![image](https://raw.githubusercontent.com/baojiweicn/Thinking-in-Coroutines/master/imgs/explain_of_asyncio.png)

## Exception

In [ ]:
import asyncio
import datetime


async def anything(i):
    print(i, datetime.datetime.now())
    await asyncio.sleep(i)


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    task = loop.create_task(anything('g'))
    try:
        result = loop.run_until_complete(task)
    except TypeError:
        print('Type error: ', task.exception())
    else:
        print(*result)
    finally:
        loop.close()
        print('close', datetime.datetime.now())

## Task Result

In [ ]:
import asyncio
import datetime


async def anything(i):
    print(i, datetime.datetime.now())
    await asyncio.sleep(i)


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    task = loop.create_task(anything(3))
    try:
        result = loop.run_until_complete(task)
        print(*result)
    finally:
        loop.close()
        print('close', datetime.datetime.now())

In [ ]:
# -*- coding: utf-8 -*-

import asyncio
import datetime


async def anything(i):
    print(i, datetime.datetime.now())
    await asyncio.sleep(i)


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    tasks = [loop.create_task(anything(i)) for i in range(1,4)]
    try:
        loop.run_until_complete(asyncio.wait(tasks))
        for task in tasks:
            print(*task.result())
    finally:
        loop.close()
        print('close', datetime.datetime.now())

## Usages

### 1. Flask Video Stream

In [ ]:
from flask import Flask, render_template, Response
from camera import Camera

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

def gen(camera):
    while True:
        frame = camera.get_frame()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/video_feed')
def video_feed():
    return Response(gen(Camera()),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True)

### 2. websockets